# Low-Rank Matrix Completion

In [5]:
import numpy as np
import pandas as pd
import scipy as sp

import time
from numpy import count_nonzero
from numpy.linalg import norm
from numpy.linalg import matrix_rank
from scipy.sparse.linalg import svds
np.random.seed(1400)

In [6]:
def sparsity(A):
    return 1.0 - count_nonzero(A)/A.size

In [7]:
n, k = 1000, 10
# random rank-10 matrix normalized to have nuclear norm 1
U = np.random.normal(0, 1, (n, k))
U = np.linalg.qr(U)[0]
S = np.diag(np.random.uniform(0, 1, k))
S /= np.sum(S)
A = U.dot(S.dot(U.T))

# A = A*1000+2

In [8]:
# r=[1,2,3,4,5]
# A=np.random.choice(r,(n,n))

# print(A.shape) 
# print(matrix_rank(A))
# print(norm(A,2))
# print(norm(A,'nuc'))

In [9]:
# Bulding a mask by passing the desired density
def make_mask(A, density):
    """Randomly zero out entries of the input ones matrix."""
    O = np.ones(A.shape)
    B = np.random.uniform(0, 1, O.shape)
    O[B > density] = 0
    return O

O = make_mask(A , 1)

In [701]:
# make A more sparse and put it into C by density that is the percentage of nonzero entries
# def make_sparse(A, density):
#     """Randomly zero out entries of the input matrix."""
#     C = np.matrix.copy(A)
#     B = np.random.uniform(0, 1, C.shape)
#     C[B > density] = 0
#     return C, B <= density

In [16]:
# multiply A to O element-wise to punch holes in A 
Y = A*O
Y

array([[ 1.53686480e-03,  2.87902608e-04, -8.70043587e-04, ...,
        -3.92201273e-05,  3.02243865e-04, -3.75075031e-04],
       [ 2.87902608e-04,  9.43788663e-04, -4.46779089e-04, ...,
         2.84071821e-04,  6.93277675e-05, -8.07776654e-04],
       [-8.70043587e-04, -4.46779089e-04,  1.10878207e-03, ...,
        -3.71902858e-04,  2.38578964e-04,  4.07192477e-04],
       ...,
       [-3.92201273e-05,  2.84071821e-04, -3.71902858e-04, ...,
         4.39590422e-04, -3.64539971e-04, -1.62346675e-04],
       [ 3.02243865e-04,  6.93277675e-05,  2.38578964e-04, ...,
        -3.64539971e-04,  1.23868622e-03, -5.60164537e-04],
       [-3.75075031e-04, -8.07776654e-04,  4.07192477e-04, ...,
        -1.62346675e-04, -5.60164537e-04,  1.76523653e-03]])

In [10]:
# Real Data
data = pd.read_csv('movielens100k.csv', names=['user id', 'item id', 'rating', 'timestamp'])
M = pd.pivot_table(data, values='rating', index='user id', columns='item id').values
m, n = M.shape
M = np.nan_to_num(M)

In [703]:
# density = 0.1
# C = np.matrix.copy(A)
# B = np.random.uniform(0, 1, C.shape)
# Because the arrays of B are taken from a uniform [0,1) so the 90% of the arrays are above 0.1
# So when we want a 10 percent density we make the
# آریه های متناظر درون ماتریس بی که بالای 0.1 هستند رو توی سی، صفر قرار میدیم.
# the code down here bringing out the 
# C[B > density]
# np.ones((3,3))

In [11]:
# we need a starting point X0 that its in the domain, we generate a random point and then project it into the domain
def simplex_projection(s):
    """Projection onto the unit simplex."""
    if np.sum(s) <=1 and np.alltrue(s >= 0):
        return s
    # Code taken from https://gist.github.com/daien/1272551
    # get the array of cumulative sums of a sorted (decreasing) copy of v
    u = np.sort(s)[::-1]
    cssv = np.cumsum(u)
    # get the number of > 0 components of the optimal solution
    rho = np.nonzero(u * np.arange(1, len(u)+1) > (cssv - 1))[0][-1]
    # compute the Lagrange multiplier associated to the simplex constraint
    theta = (cssv[rho] - 1) / (rho + 1.0)
    # compute the projection by thresholding v using theta
    return np.maximum(s-theta, 0)

def nuclear_projection(A):
    """Projection onto nuclear norm ball."""
    U, s, V = np.linalg.svd(A, full_matrices=False)
    s = simplex_projection(s)
    return U.dot(np.diag(s).dot(V))

In [705]:
X0 = np.random.normal(0,1,(n,n))
X0 = nuclear_projection(X0.dot(X0.T))

In [12]:
def mc_objective(Y, O, X):
    """Matrix completion objective.
        Y : the to be completed matrix ماتریس ناقص شده با سوراخ های ما 
        X*O : آرایه هایی از ماتریس داده واقعی که مشاهده شده اند
    """
    return 0.5 * np.linalg.norm(Y-np.multiply(X, O), 'fro')**2

def mc_gradient(Y, O, X):
    """Gradient of matrix completion objective."""
    return np.multiply(X, O) - Y

In [32]:
def power_method(Z,num_steps=5):
    """
    Power method : Computes the top left and right eigen vectors.
    Parameters:
    ------------
        Z : array {m,n},
            input matrix,
        num_steps : integer,
            optional, number of iteration of the algorithm.
    Returns :
    ------------
        u,v : (m,), (n,)
            two arrays representing approximated unit length top
            left and right eigen vector of Z.
    """
    #pick a random unit vector x and compute A.T.x/||A.T.x||
    m, n = Z.shape
    u = np.random.normal(0,1,m)
    u = u /norm(u,2)
    v = Z.T.dot(u)
    v = v /norm(v, 2)
    for i in range(num_steps):
        u = Z.dot(v)
        u = u/norm(u,2)
        v = Z.T.dot(u)
        v = v/norm(v,2)
    return u,v

TypeError: mc_gradient() missing 2 required positional arguments: 'O' and 'X'

In [41]:
u, s, v = svds(-Y, k=1, which='LM')
u

array([[ 0.05009817],
       [ 0.01158281],
       [-0.02245449],
       [ 0.01024854],
       [ 0.00802561],
       [-0.00957649],
       [-0.08170323],
       [-0.05702841],
       [ 0.01100752],
       [ 0.03889061],
       [-0.00635224],
       [-0.00683357],
       [ 0.02164414],
       [-0.02230109],
       [-0.02694501],
       [-0.02492126],
       [ 0.00867756],
       [-0.00826431],
       [-0.01242309],
       [ 0.0069327 ],
       [-0.00798674],
       [ 0.00762308],
       [ 0.06579516],
       [-0.0197406 ],
       [ 0.02004707],
       [-0.01652232],
       [-0.01476643],
       [ 0.04067296],
       [-0.02834631],
       [ 0.04599638],
       [-0.0075389 ],
       [-0.04850778],
       [-0.01274284],
       [ 0.0016767 ],
       [ 0.02334933],
       [-0.0326873 ],
       [ 0.00216366],
       [ 0.00231382],
       [-0.02600415],
       [-0.00029691],
       [ 0.02389754],
       [-0.0093613 ],
       [ 0.00031212],
       [-0.06890225],
       [-0.00048491],
       [ 0

In [39]:
u, v = power_method(-Y)
u.reshape(1000,1)

array([[-4.76352970e-02],
       [ 2.85341485e-03],
       [ 1.51831325e-02],
       [-4.98228757e-04],
       [ 6.67941937e-04],
       [-1.02777764e-03],
       [ 7.08229686e-02],
       [ 5.95417013e-02],
       [-1.20913169e-02],
       [-4.19152828e-02],
       [ 1.16408369e-02],
       [-5.26505095e-04],
       [-4.69410702e-03],
       [ 2.73251209e-02],
       [ 3.16838836e-02],
       [ 1.90027396e-02],
       [-7.58134693e-03],
       [ 1.37297475e-02],
       [ 1.01256823e-02],
       [-4.99694628e-03],
       [ 1.53742654e-02],
       [-8.96164424e-03],
       [-6.56392662e-02],
       [ 3.74092515e-02],
       [-2.08523112e-02],
       [ 1.37026730e-02],
       [ 6.14301468e-03],
       [-2.40319602e-02],
       [ 2.62134124e-02],
       [-5.15301392e-02],
       [-9.40692442e-04],
       [ 4.27785153e-02],
       [ 4.78084723e-03],
       [-9.16733723e-04],
       [-1.13003864e-02],
       [ 3.78473128e-02],
       [-1.17481270e-02],
       [-7.49643764e-03],
       [ 2.4

In [708]:
def mc_oracle(Y, O, X):
    """Linear optimization oracle for matrix completion."""
    G = mc_gradient(Y, O, X)
    u, v = power_method(-G)
    return u.reshape((len(u), 1)).dot(v.reshape((1,len(v))))

In [709]:
def conditional_gradient(initial, steps, almo):
    """
    parameters:
    ------------
        initial: array,
            initial starting point
        steps : list of numbers,
            step size , output of line search.
        almo : function, the argument that
        optimize the linear function(that defined by the gradient)
            the direction in which we gain the minimize value of 
            mapping points to points,
            implements liniear optimization oracle for the objective.
    Returns:
    ------------
        List of points, computed by the algorithm.
        the points are the input that gradually 
        increase the amount of objective function.
        xs[-1] is the last computed point in the xs list,
        that is collectig every computed point of the algorith.
        
    """
    xs = [initial]
    for step in steps:
        xs.append( xs[-1] + step*( almo(xs[-1]) - xs[-1]))
    return xs[-1]

In [710]:
almo = lambda X: mc_oracle(Y,O,X)
steps = [2.0/(k+2.0) for k in range(1,40)]
Xs = conditional_gradient( X0, steps, almo )

In [711]:
print("About X0") 
print("shape:",X0.shape) 
print('rank:' , matrix_rank(X0))
print('norm fro' , norm(X0,'fro'))
print('norm nuc:', norm(X0,'nuc'))
print('max: ',X0.max())
print('min: ',X0.min())
print('mean: ',X0.mean())

About X0
shape: (1000, 1000)
rank: 1
norm fro 0.9999999999999982
norm nuc: 1.0000000000000089
max:  0.00998976549977452
min:  -0.008980984360557921
mean:  3.372658565177119e-06


In [712]:
print("About Xs") 
print("shape:",Xs.shape) 
print('rank:' , matrix_rank(Xs))
print('norm fro' , norm(Xs,'fro'))
print('norm nuc:', norm(Xs,'nuc'))
print('max: ',Xs.max())
print('min: ',Xs.min())
print('mean: ',Xs.mean())

About Xs
shape: (1000, 1000)
rank: 11
norm fro 0.34685809916268107
norm nuc: 0.8846740289910843
max:  0.0028431616645700638
min:  -0.002177289013323378
mean:  1.5826332241218227e-06


In [725]:
Xs2 = Xs*1000+2
print("About Xs2") 
print("shape:",Xs2.shape) 
print('rank:' , matrix_rank(Xs2))
print('norm fro' , norm(Xs2,'fro'))
print('norm nuc:', norm(Xs2,'nuc'))
print('max: ',Xs2.max())
print('min: ',Xs2.min())
print('mean: ',Xs2.mean())

About Xs2
shape: (1000, 1000)
rank: 12
norm fro 2031.4135654394054
norm nuc: 2884.6620579606697
max:  4.843161664570063
min:  -0.17728901332337843
mean:  2.0015826332241224


In [723]:
print("About A") 
print("shape:",A.shape) 
print('rank:' , matrix_rank(A))
print('norm fro' , norm(A,'fro'))
print('norm nuc:', norm(A,'nuc'))
print('max: ',A.max())
print('min: ',A.min())
print('mean: ',A.mean())

About A
shape: (1000, 1000)
rank: 10
norm fro 0.37779739128222817
norm nuc: 1.0000000000000033
max:  0.0030599708090525857
min:  -0.002269335422884872
mean:  1.8413448445214025e-06


In [724]:
A2 = A*1000+2
print("About A2") 
print("shape:",A2.shape) 
print('rank:' , matrix_rank(A2))
print('norm fro' , norm(A2,'fro'))
print('norm nuc:', norm(A2,'nuc'))
print('max: ',A2.max())
print('min: ',A2.min())
print('mean: ',A2.mean())

About A2
shape: (1000, 1000)
rank: 11
norm fro 2037.178501810222
norm nuc: 3000.0000000000173
max:  5.059970809052586
min:  -0.2693354228848719
mean:  2.001841344844522


In [714]:
mean_error = np.sum(abs(A-Xs))/n**2
mean_error

4.107053841863576e-05

In [715]:
def calc_unobserved_rmse(A , Xs, O):
    """
    Calculate RMSE on all unobserved entries in mask, for true matrix A.
    Parameters
    ----------
    A : 
    Xs :
    O : mask : m x n array
        matrix with entries zero (if missing) or one (if present)
    Returns:
    --------
    rmse : float
        root mean squared error over all unobserved entries
    """
    pred = np.multiply(Xs, (1 - O))
    truth = np.multiply(A, (1 - O))
    cnt = np.sum(1 - O)
    return (np.linalg.norm(pred - truth, "fro") ** 2 / cnt) ** 0.5


In [716]:
calc_unobserved_rmse(A , Xs, O)

<ipython-input-715-70b18d480cb3>:18: RuntimeWarning: invalid value encountered in double_scalars
  return (np.linalg.norm(pred - truth, "fro") ** 2 / cnt) ** 0.5


nan

In [717]:
def calc_observed_rmse(A , Xs, O):
    """
    Calculate RMSE on all unobserved entries in mask, for true matrix A.
    Parameters
    ----------
    A : 
    Xs :
    O : mask : m x n array
        matrix with entries zero (if missing) or one (if present)
    Returns:
    --------
    rmse : float
        root mean squared error over all observed entries
    """
    pred = np.multiply(Xs, O)
    truth = np.multiply(A, O)
    cnt = np.sum(O)
    return (np.linalg.norm(pred - truth, "fro") ** 2 / cnt) ** 0.5


In [718]:
calc_observed_rmse(A , Xs, O)

5.2913850249269347e-05

In [719]:
def calc_rmse(A , Xs):
    """
    Calculate RMSE on all entries in Xs, for true matrix A.
    Parameters
    ----------
    A : 
    Xs :
    Returns:
    --------
    rmse : float
        root mean squared error over all entries
    """

    return (np.linalg.norm(Xs-A, "fro") ** 2 / n**2) ** 0.5

In [720]:
calc_rmse(A , Xs)

5.2913850249269347e-05

In [19]:
import os
import numpy as np
import time
import scipy.sparse as ss

path_prefix = 'F:\1m'


def dataloader(dataset='ratings'):
#     start = time.clock()
    fname = 'F:\1m\ratings.dat'
    max_uid = 0
    max_vid = 0
    users = []
    movies = []
    ratings = []
    first_line_flag = True
    with open(fname) as f:
        for line in f:
            tks = line.strip().split('::')
            # tks = m
            if first_line_flag:
                max_uid = int(tks[0])
                max_vid = int(tks[1])
                first_line_flag = False
                continue
            max_uid = max(max_uid, int(tks[0]))
            max_vid = max(max_vid, int(tks[1]))
            users.append((int(tks[0]) - 1))
            movies.append(int(tks[1]) - 1)
            ratings.append(int(tks[2]))
    M_original = ss.csr_matrix(
        (ratings, (users, movies)), shape=(max_uid, max_vid))
#     print("loading data time:%f" % (time.clock()-start))
    return M_original


In [20]:
M_original = dataloader()

OSError: [Errno 22] Invalid argument: 'F:\x01m\ratings.dat'

In [14]:
M_original

<function __main__.dataloader(dataset='ratings')>